In [5]:
%load_ext autoreload
%autoreload 2
import HammingCode
import matplotlib.pyplot as plt
import numpy as np
from scipy.special import erfc
import NeuralNetworkCorrection as network
import Autoencoder
import Utils

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [10]:
variancia = 1
code_rate = 4/7
# amostra
Eb_dB_values = np.arange(3, 16, 1)

# Gerar dados de treinamento
training_sample_length = 10 ** 3
epoches = 300
batch_size = 16

In [11]:
training_data = {
    'original': [],
    'noisy': [],
    'info': []
}

training_information = network.get_training_data_set(training_sample_length)

for eb_db in Eb_dB_values:
    info = network.generate_data_for_training(training_information, eb_db, variancia)
    training_data['noisy'].extend(info['split_noisy_original_data_15_11'])
    training_data['original'].extend(info['split_original_data_15_11'])

training_data['noisy'] = np.array(training_data['noisy'])
training_data['original'] = np.array(training_data['original'])

In [12]:
# Criar e treinar a rede neural
model = network.train_neural_network(training_data, epoches, batch_size)

Epoch 1/300
67/67 [==============================] - 2s 11ms/step - loss: 0.6144 - accuracy: 0.1006 - val_loss: 0.5601 - val_accuracy: 0.1849
Epoch 2/300
67/67 [==============================] - 0s 6ms/step - loss: 0.5250 - accuracy: 0.1945 - val_loss: 0.5403 - val_accuracy: 0.0924
Epoch 3/300
67/67 [==============================] - 0s 6ms/step - loss: 0.4971 - accuracy: 0.1504 - val_loss: 0.5047 - val_accuracy: 0.2773
Epoch 4/300
67/67 [==============================] - 0s 6ms/step - loss: 0.4802 - accuracy: 0.1720 - val_loss: 0.4975 - val_accuracy: 0.3109
Epoch 5/300
67/67 [==============================] - 0s 6ms/step - loss: 0.4773 - accuracy: 0.1447 - val_loss: 0.5083 - val_accuracy: 0.1597
Epoch 6/300
67/67 [==============================] - 0s 6ms/step - loss: 0.4652 - accuracy: 0.1776 - val_loss: 0.4967 - val_accuracy: 0.2101
Epoch 7/300
67/67 [==============================] - 0s 6ms/step - loss: 0.4628 - accuracy: 0.1551 - val_loss: 0.5019 - val_accuracy: 0.3193
Epoch 8/300


In [46]:
autoencoder = Autoencoder.create_and_train_auto_encoder(training_data, epoches, batch_size)

Epoch 1/30
7387/7387 [==============================] - 35s 5ms/step - loss: 0.5532 - accuracy: 0.1470 - val_loss: 0.5153 - val_accuracy: 0.0856
Epoch 2/30
7387/7387 [==============================] - 32s 4ms/step - loss: 0.5116 - accuracy: 0.1438 - val_loss: 0.5080 - val_accuracy: 0.0609
Epoch 3/30
7375/7387 [============================>.] - ETA: 0s - loss: 0.5039 - accuracy: 0.1205

KeyboardInterrupt: 

In [6]:
# Gerar dados de teste
amostras_ruidosas = []
amostras_originais = []
ML_result = []
tradicional_result = []
autoencoder_result = []


In [7]:
test_data = {
    'original': [],
    'info': []
}
test_sample_length = 10 ** 3
test_information = network.get_training_data_set(test_sample_length)

for eb_db in Eb_dB_values:
    info = network.generate_data_for_training(test_information, eb_db, variancia)
    test_data['info'].append(info)
    test_data['original'] = info['original_data']

In [8]:
i = 0
autoencoder_result.append(network.decode_and_correct(test_data['info'][i]['split_noisy_original_data_15_11'], autoencoder))
ML_result.append(network.decode_and_correct(test_data['info'][i]['split_noisy_original_data_15_11'], model))

In [9]:
i = 1
autoencoder_result.append(network.decode_and_correct(test_data['info'][i]['split_noisy_original_data_15_11'], autoencoder))
ML_result.append(network.decode_and_correct(test_data['info'][i]['split_noisy_original_data_15_11'], model))

In [10]:
i = 2
autoencoder_result.append(network.decode_and_correct(test_data['info'][i]['split_noisy_original_data_15_11'], autoencoder))
ML_result.append(network.decode_and_correct(test_data['info'][i]['split_noisy_original_data_15_11'], model))

In [11]:
i = 3
autoencoder_result.append(network.decode_and_correct(test_data['info'][i]['split_noisy_original_data_15_11'], autoencoder))
ML_result.append(network.decode_and_correct(test_data['info'][i]['split_noisy_original_data_15_11'], model))


In [12]:
i = 4
autoencoder_result.append(network.decode_and_correct(test_data['info'][i]['split_noisy_original_data_15_11'], autoencoder))
ML_result.append(network.decode_and_correct(test_data['info'][i]['split_noisy_original_data_15_11'], model))


In [13]:
i = 5
autoencoder_result.append(network.decode_and_correct(test_data['info'][i]['split_noisy_original_data_15_11'], autoencoder))
ML_result.append(network.decode_and_correct(test_data['info'][i]['split_noisy_original_data_15_11'], model))

In [14]:
i = 6
autoencoder_result.append(network.decode_and_correct(test_data['info'][i]['split_noisy_original_data_15_11'], autoencoder))
ML_result.append(network.decode_and_correct(test_data['info'][i]['split_noisy_original_data_15_11'], model))


In [15]:
i = 7
autoencoder_result.append(network.decode_and_correct(test_data['info'][i]['split_noisy_original_data_15_11'], autoencoder))
ML_result.append(network.decode_and_correct(test_data['info'][i]['split_noisy_original_data_15_11'], model))


In [16]:
i = 8
autoencoder_result.append(network.decode_and_correct(test_data['info'][i]['split_noisy_original_data_15_11'], autoencoder))
ML_result.append(network.decode_and_correct(test_data['info'][i]['split_noisy_original_data_15_11'], model))


In [17]:
i = 9
autoencoder_result.append(network.decode_and_correct(test_data['info'][i]['split_noisy_original_data_15_11'], autoencoder))
ML_result.append(network.decode_and_correct(test_data['info'][i]['split_noisy_original_data_15_11'], model))


In [18]:
i = 10
autoencoder_result.append(network.decode_and_correct(test_data['info'][i]['split_noisy_original_data_15_11'], autoencoder))
ML_result.append(network.decode_and_correct(test_data['info'][i]['split_noisy_original_data_15_11'], model))


In [19]:
i = 11
autoencoder_result.append(network.decode_and_correct(test_data['info'][i]['split_noisy_original_data_15_11'], autoencoder))
ML_result.append(network.decode_and_correct(test_data['info'][i]['split_noisy_original_data_15_11'], model))


In [20]:
i = 12
autoencoder_result.append(network.decode_and_correct(test_data['info'][i]['split_noisy_original_data_15_11'], autoencoder))
ML_result.append(network.decode_and_correct(test_data['info'][i]['split_noisy_original_data_15_11'], model))


In [21]:
#for i in test_data['info']:
#    tradicional_result.append(HammingCode.decode_sample(i['split_encoded_data']))

In [22]:
BER_tradicional = []
BER_autoencoder = []
BER_ML = []
for i in range(0, len(autoencoder_result)):
    print(len(ML_result[i]))
    BER_ML.append(np.sum(np.array(ML_result[i]).astype(int) != test_data['original']) / len(test_data['original']))
    #BER_tradicional.append(np.sum(np.array(tradicional_result[i]).astype(int) != test_data['original']) / len(test_data['original']))
    BER_autoencoder.append(np.sum(np.array(autoencoder_result[i]).astype(int) != test_data['original']) / len(test_data['original']))
    
# Calcular a BER teórica
Eb_coded = np.array([eb_db + 10*np.log10(code_rate) for eb_db in Eb_dB_values])
Eb_values = 10**(Eb_coded/10)
BER_teorica = 0.5 * erfc(np.sqrt(Eb_values))

1001


ValueError: operands could not be broadcast together with shapes (1001,) (1000,) 

In [ ]:

# Plotar as taxas de erro de bit simuladas e teóricas
plt.figure(figsize=(10, 6))
plt.semilogy(Eb_coded, BER_tradicional, linestyle = '--', label='Codificado')
plt.semilogy(Eb_coded, BER_autoencoder, linestyle = '--', label='Autoencoder')
plt.semilogy(Eb_coded, BER_ML, linestyle = '--', label='MLP')
plt.semilogy(Eb_coded, BER_teorica, linestyle='--', label='Teórica')
plt.xlabel('Eb/No (dB)')
plt.ylabel('BER')
plt.grid(True, which='both')
plt.legend()
plt.title('BER x Eb/No')
plt.show()